In [96]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import numpy as np
from scipy.stats import uniform, randint
from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [95]:
data = pd.read_csv('../data/Breast_Cancer.csv')
print(f'Original dataset: {data.shape}')

def check_binary(df):
    return [column for column in df.columns if df[column].isin(['Yes','No']).any()]

def check_pseudobi(df):
    return [column for column in df.columns if df[column].isin(np.arange(0.0, 5.0)).any()]

def check_marital(df):
    return [column for column in df.columns if df[column].isin(np.unique(data['marital'])).any()]

def getcolumns_rate(dataframe, rate = 0.25): 
  less_than_50pct_nonnull = (dataframe.isnull().sum() / dataframe.shape[0]) < rate
  return dataframe.columns[less_than_50pct_nonnull]

def get_objectColumns(dataframe, type = 'object'):
    return list(dataframe.select_dtypes(include = type).columns)

def variance_inflation(df):
    vif = pd.DataFrame()
    vif['VIF'] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
    vif['Features'] = data.columns
    vif.sort_values(by = 'VIF', ascending = False)

def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))

def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                results['mean_test_score'][candidate], results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

remove_columns = getcolumns_rate(data)
data = data[remove_columns]
data = data.dropna()
encoder = LabelEncoder()

data['bdate'] = data['bdate'].str[-4:]
data['age']   = 2566 - data['bdate'].astype('int')
data = data.drop('bdate', axis = 1)

encode_columns = get_objectColumns(data)
for column in encode_columns:
    data[column] = encoder.fit_transform(data[column])

data['noova'] = data['noova'].replace([9.0], [3.0], inplace = False)
encode_columns = ['nobreast', 'nosecon', 'noova', 'inj', 'noparity']
for column in encode_columns:
    data[column] = encoder.fit_transform(data[column])

X = data.drop('diag_cancer', axis = 1).to_numpy()
y = data['diag_cancer'].to_numpy()

Original dataset: (15718, 46)


In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

print('==============Gradient Boosting Classifier=================')
clf = GradientBoostingClassifier(n_estimators = 100, learning_rate = 1.0, max_depth = 1, random_state=0)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))

print('=============XGBoost Classifier===============')
xgb_model = xgb.XGBClassifier(n_estimators=100,eta=0.3, gamma=0.5, random_state=0)
xgb_model.fit(X_train, y_train)
print(xgb_model.score(X_test, y_test))
y_pred_xgb = xgb_model.predict(X_test)
print(confusion_matrix(y_test, y_pred_xgb))

==============Gradient Boosting Classifier=================
0.9947072444591465
[[  18   10]
 [   6 2989]]
=============XGBoost Classifier===============
0.9943764472378432
[[  15   13]
 [   4 2991]]


In [98]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", n_estimators=20, random_state=42, eval_metric=["auc", "error", "error@0.6"])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)])
y_pred = xgb_model.predict(X_test)

[0]	validation_0-auc:0.99897	validation_0-error:0.00238	validation_0-error@0.6:0.00450
[1]	validation_0-auc:0.99916	validation_0-error:0.00265	validation_0-error@0.6:0.00371
[2]	validation_0-auc:0.99908	validation_0-error:0.00265	validation_0-error@0.6:0.00265
[3]	validation_0-auc:0.99901	validation_0-error:0.00291	validation_0-error@0.6:0.00238
[4]	validation_0-auc:0.99906	validation_0-error:0.00291	validation_0-error@0.6:0.00291
[5]	validation_0-auc:0.99902	validation_0-error:0.00291	validation_0-error@0.6:0.00291
[6]	validation_0-auc:0.99897	validation_0-error:0.00291	validation_0-error@0.6:0.00291
[7]	validation_0-auc:0.99903	validation_0-error:0.00318	validation_0-error@0.6:0.00265
[8]	validation_0-auc:0.99905	validation_0-error:0.00318	validation_0-error@0.6:0.00291
[9]	validation_0-auc:0.99910	validation_0-error:0.00318	validation_0-error@0.6:0.00318
[10]	validation_0-auc:0.99912	validation_0-error:0.00318	validation_0-error@0.6:0.00291
[11]	validation_0-auc:0.99912	validation_0

In [99]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric="auc")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
xgb_model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_test, y_test)], verbose=False)
y_pred = xgb_model.predict(X_test)
print(confusion_matrix(y_test, y_pred))

[[  23    3]
 [   7 3745]]
